In [12]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings


if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

model = "gpt-3.5-turbo"

llm = ChatOpenAI(temperature=0.5, model=model, max_tokens=4096)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [13]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="real-state",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db_txt",  # Where to save data locally, remove if not necessary
)

## Retriever

In [14]:
retriever = vector_store.as_retriever()

## Prompt template

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

template = """Responde las siguientes preguntas con base en los siguientes contexto:
Eres un asistente virtual especializado en una agencia inmobiliaria. Tu tarea es proporcionar información precisa y útil sobre los proyectos inmobiliarios disponibles. Utiliza únicamente la información proporcionada en los documentos para responder.

Proyectos disponibles:
1. **Proyecto A**: Residencial "Bosque Verde"
2. **Proyecto B**: Comercial "Plaza Sol"
3. **Proyecto C**: Residencial de lujo "Altos del Mar"

Instrucciones:
- Responde de manera clara y concisa.
- Si no tienes información suficiente, indica que no cuentas con los detalles necesarios.
- Prioriza la información sobre ubicación, precios, características y disponibilidad.
- El año actual es 2025.

{context}
"""
prompt_template = ChatPromptTemplate([
    ("system", template),
    ("user", "{question}")
])

## RAG Chain

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [17]:
response = rag_chain.invoke("dame precios?")
print(response)

Claro, aquí tienes la información de los precios de los proyectos disponibles:

1. **Proyecto A - Residencial "Bosque Verde"**: Los precios van desde $150,000 hasta $300,000.
2. **Proyecto B - Comercial "Plaza Sol"**: No se especifican los precios en el documento proporcionado.
3. **Proyecto C - Residencial de lujo "Altos del Mar"**: Los precios van desde $500,000 hasta $1,000,000.


## RAG Graph

In [18]:
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"],k=3)
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt_template.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [19]:
response = graph.invoke({"question": "hablame todo acerca del proyecto A, dame precios"})
print(response["answer"])

El proyecto A es el residencial "Bosque Verde". Se encuentra ubicado en la zona sur de la ciudad, cerca de parques y áreas verdes. Ofrece apartamentos de 2 y 3 habitaciones, con amplias áreas comunes, piscina y gimnasio. En cuanto a precios, los apartamentos de 2 habitaciones tienen un valor desde $150,000, mientras que los de 3 habitaciones parten desde $200,000. Para más detalles sobre disponibilidad y formas de pago, te recomendaría contactar directamente a la agencia inmobiliaria a cargo del proyecto.
